In [42]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA

import tensorflow_probability as tfp

from tensorflow_probability import bijectors as tfb
from tensorflow_probability import distributions as tfd

ModuleNotFoundError: No module named 'tensorflow_probability'

In [2]:
df = pd.read_csv('./Data/clientsSynthAgg.csv')

In [7]:
df = df.dropna()
df = df.drop(columns=['CD_ESTADO','CD_POSTAL','CD_SEXO', 'NU_CTE_COD'])

In [8]:
df.head()

,EDAD,acc,caminar,ver,escuchar,hablar,cuidado,recordar,mental,mean_AUTOSERVICIOS_4,...,"count_FOTOGRAFIA, ARTE Y ANUNCIOS COMERCIALES,(ARTES Y ANUNCIOS_9",count_OSTEOPATAS_5,"count_CONTRATISTAS TECHOS, ESTRUCTURAS Y LAMINAS_6","count_CONTRATISTAS (YESO) (CANTERA Y MAMPOSTERIA) (TEJAS, PISOS Y_7","count_VENTA DE TOLDOS, (VENTA DE TIENDAS DE CAMPANA),_8",count_HAMPTON INN HOTELS_4,count_BAILE SALONES Y ESCUELAS DE BAILE_9,count_SERVICIO GRUAS Y TRASLADOS_6,count_VENTAS DE SALDOS_5,count_UNITED AIRLINES_5
0,68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6574.390000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,342.985000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,338.830000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,79,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1281.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.557692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# PCA

In [34]:
pcaDF = df.drop(columns=['acc','caminar','ver','escuchar','hablar','cuidado','recordar','mental'])
X = pcaDF.values

In [37]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled

array([[ 0.91667591,  8.25839963,  0.82977197, ..., -0.01420048,
        -0.01420048, -0.01420048],
       [-0.54462558, -0.01501019,  4.9935019 , ..., -0.01420048,
        -0.01420048, -0.01420048],
       [-0.54462558, -0.02052677, -0.37179825, ..., -0.01420048,
        -0.01420048, -0.01420048],
       ...,
       [-0.1634165 , -0.47038991, -0.37179825, ..., -0.01420048,
        -0.01420048, -0.01420048],
       [ 0.02718805,  0.87160068,  0.51492202, ..., -0.01420048,
        -0.01420048, -0.01420048],
       [ 0.28132744,  1.46870086,  0.8482453 , ..., -0.01420048,
        -0.01420048, -0.01420048]])

In [40]:
pca_30 = PCA(n_components=30, random_state=1337)
pca_30.fit(X_scaled)
X_pca_30 = pca_30.transform(X_scaled)

In [41]:
pca_30.explained_variance_ratio_ * 100

array([1.14466424, 0.5412765 , 0.49745046, 0.4604444 , 0.44164034,
       0.4290547 , 0.42583807, 0.41384   , 0.40894061, 0.40432404,
       0.39139967, 0.38646878, 0.38030763, 0.37834029, 0.37411945,
       0.36952569, 0.36433286, 0.36173602, 0.35480181, 0.34732977,
       0.3464548 , 0.34268253, 0.33805874, 0.33337704, 0.32870023,
       0.32647305, 0.3221755 , 0.32114664, 0.31642577, 0.31062588])

# TF PCA

In [56]:
pcaDF = df.drop(columns=['caminar','ver','escuchar','hablar','cuidado','recordar','mental'])
pcaDF = StandardScaler().fit_transform(pcaDF)
pcaDF_acc = PCA(n_components=2)
principalComponents_acc = pcaDF_acc.fit_transform(pcaDF)

# TF

In [96]:
targetModels = ['caminar','ver','escuchar','hablar','cuidado','recordar','mental']
for targetModel in targetModels: 
    print('MODEL {}'.format(targetModel))
    X = df.drop(columns=['acc','caminar','ver','escuchar','hablar','cuidado','recordar','mental', 'EDAD'])
    y = df[targetModel]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, random_state=3376
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    tf.random.set_seed(1337)
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        loss=tf.keras.losses.binary_crossentropy,
        optimizer=tf.keras.optimizers.Adam(lr=0.03),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )

    history = model.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback],)
    predictions = model.predict(X_test_scaled)
    prediction_classes = [
        1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
    ]
    print(Counter(prediction_classes).values())
    print(confusion_matrix(y_test, prediction_classes))
    print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
    print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
    print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')
    tf.keras.models.save_model(
        model,
        'models/{}'.format(targetModel),
        overwrite=True,
        include_optimizer=True,
        save_format=None,
        signatures=None,
        options=None
    )

MODEL caminar
Epoch 1/100


/home/psyrax/anaconda3/envs/tfgpu39/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


124/124 [==============================] - 1s 2ms/step - loss: 2.6335 - accuracy: 0.9020 - precision: 0.0580 - recall: 0.0300
Epoch 2/100
124/124 [==============================] - 0s 2ms/step - loss: 0.9391 - accuracy: 0.9259 - precision: 0.0345 - recall: 0.0037 
Epoch 3/100
124/124 [==============================] - 0s 2ms/step - loss: 0.3015 - accuracy: 0.9307 - precision: 0.1667 - recall: 0.0075 
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 0.2494 - accuracy: 0.9335 - precision: 1.0000 - recall: 0.0112
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 0.2450 - accuracy: 0.9335 - precision: 1.0000 - recall: 0.0112  
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 0.2458 - accuracy: 0.9335 - precision: 1.0000 - recall: 0.0112
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 1.0645 - accuracy: 0.9327 - precision: 0.5000 - recall: 0.0037
Epoch 8/100
124/124 [=====================

/home/psyrax/anaconda3/envs/tfgpu39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: models/caminar/assets
MODEL ver
Epoch 1/100


...

INFO:tensorflow:Assets written to: models/recordar/assets
MODEL mental
Epoch 1/100


/home/psyrax/anaconda3/envs/tfgpu39/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


124/124 [==============================] - 1s 2ms/step - loss: 3.8828 - accuracy: 0.9723 - precision: 0.0175 - recall: 0.0182
Epoch 2/100
124/124 [==============================] - 0s 2ms/step - loss: 0.1410 - accuracy: 0.9861 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/100
124/124 [==============================] - 0s 2ms/step - loss: 0.1023 - accuracy: 0.9861 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 0.1693 - accuracy: 0.9861 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 0.0732 - accuracy: 0.9861 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 0.1550 - accuracy: 0.9861 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 0.3133 - accuracy: 0.9846 - precision: 0.0000e+00 - recall: 0.0000e+0

/home/psyrax/anaconda3/envs/tfgpu39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: models/mental/assets


In [107]:
X = df.drop(columns=['acc','caminar','ver','escuchar','hablar','cuidado','recordar','mental'])
y = df['acc']
y.to_csv('y.csv', index=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=3376
)

In [100]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [101]:
tf.random.set_seed(1337)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback],)

Epoch 1/100


/home/psyrax/anaconda3/envs/tfgpu39/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


124/124 [==============================] - 1s 2ms/step - loss: 1.5286 - accuracy: 0.8117 - precision: 0.1000 - recall: 0.0150
Epoch 2/100
124/124 [==============================] - 0s 2ms/step - loss: 0.4843 - accuracy: 0.8309 - precision: 0.1667 - recall: 0.0015   
Epoch 3/100
124/124 [==============================] - 0s 2ms/step - loss: 0.4561 - accuracy: 0.8317 - precision: 0.4000 - recall: 0.0030
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 0.5064 - accuracy: 0.8319 - precision: 0.5000 - recall: 0.0015   
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 0.4559 - accuracy: 0.8317 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 0.4539 - accuracy: 0.8319 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 0.4534 - accuracy: 0.8319 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 8/100
124/

In [102]:
predictions = model.predict(X_test_scaled)

In [103]:
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

In [104]:
Counter(prediction_classes).values()

dict_values([992])

In [105]:
print(confusion_matrix(y_test, prediction_classes))

[[842   0]
 [150   0]]


In [106]:
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.85
Precision: 0.00
Recall: 0.00


/home/psyrax/anaconda3/envs/tfgpu39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
tf.keras.models.save_model(
    model,
    'models/{}'.format(targetModel),
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

2022-10-22 22:19:53.881273: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/acc/assets
